In [1]:
import psycopg2
from psycopg2 import sql
from psycopg2 import Error

In [2]:
DB_HOST = 'pavlov.ing.puc.cl'
DB_PORT = '5432'
DB_USER = 'grupo64'
DB_PASSWORD = 'grupo64'
DB_NAME = 'grupo64e2'

In [3]:
# Ruta del archivo CSV
archivo_csv_platos = '../datos e2/platos.csv'
archivo_csv_restaurantes= '../datos e2/restaurantes2.csv'

# Configuración de la conexión
# try:
conexion = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)
# Crear un cursor
cursor = conexion.cursor()

# Dropear la tabla si ya existe
drop_table_query = """
DROP TABLE IF EXISTS platos;
DROP TABLE IF EXISTS servido_en;
"""
cursor.execute(drop_table_query)
print("Tablas platos, y servido_en eliminada si existía")

# Crear la tabla (ajusta esto según tu esquema de datos)
crear_tabla_query = """
CREATE TABLE IF NOT EXISTS platos (
    id INT NOT NULL,
    nombre varchar   NOT NULL,
    descripcion varchar NOT NULL,
    disponibilidad bool   NOT NULL,
    estilo varchar   NOT NULL,
    restriccion varchar  NOT NULL,
    ingredientes varchar NOT NULL,
    porciones INT   NOT NULL,
    tiempo INT NOT NULL,
    vigente bool   NOT NULL,
    CONSTRAINT pk_plato PRIMARY KEY (
        id))
"""

crear_tabla_query_relation = """
CREATE TABLE IF NOT EXISTS servido_en (
    restaurante varchar NOT NULL,
    id_plato INT   NOT NULL,
    precio INT NOT NULL)
"""
cursor.execute(crear_tabla_query)
conexion.commit()

cursor.execute(crear_tabla_query_relation)
conexion.commit()

# Preparar el query de inserción
insercion_query = """
INSERT INTO platos (id, nombre, descripcion, disponibilidad, estilo, restriccion, ingredientes, porciones, tiempo, vigente)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""
insercion_query_relation = """
INSERT INTO servido_en (restaurante, id_plato, precio)
VALUES (%s, %s, %s);
"""

with open(archivo_csv_restaurantes, 'r') as file:
    lista_restaurante = []
    restaurante_actual = ""
    next(file)

    for linea in file:
        restaurante_actual += linea.strip()
        valores = restaurante_actual.split(';')
        if len(valores) == 8:
            lista_restaurante.append(valores)
            restaurante_actual = ""
        elif len(valores) > 8:
            print(f"Línea malformada: {restaurante_actual}")
            restaurante_actual = ""



    # Leer y procesar el archivo CSV
    with open(archivo_csv_platos, 'r') as file:
        lista_plato = []
        plato_actual = ""
        next(file)
        relations = []
        list_of_completed = []

        for linea in file:
            plato_actual += linea.strip()
            valores = plato_actual.split(';')
            if len(valores) == 13:
                lista_plato.append(valores)
                plato_actual = ""
            elif len(valores) > 13:
                print(f"Línea malformada: {plato_actual}")
                plato_actual = ""

        
        for plato in lista_plato:
            for restaurant in lista_restaurante:
                if plato[10] == restaurant[0]:
                    id_plato = plato[0]
                    precio = plato[8]
                    restaurante = restaurant[0]
                    relations.append([restaurante, id_plato, precio])
                    
                    try:
                        for relation in relations:
                            if relation not in list_of_completed:
                                cursor.execute(insercion_query_relation, (relation[0], int(relation[1]), int(relation[2])))
                                list_of_completed.append(relation)
                    except Error as e:
                            print(f"Error al insertar la línea {linea}: {e}")
                            conexion.rollback()
                            continue
            try:
                cursor.execute(insercion_query, (int(plato[0]), plato[1], plato[2], plato[3], plato[4], plato[5], plato[6], int(plato[7]), int(plato[9]),  plato[12]))
            except Error as e:
                    print(f"Error al insertar la línea {linea}: {e}")
                    conexion.rollback()
                    continue

# Confirmar la inserción
conexion.commit()

# except Error as e:
#     print(f"Error de conexión o durante la operación en la base de datos: {e}")

# finally:

#     if cursor:
#         cursor.close()
#     if conexion:
#         conexion.close()

Tablas platos, y servido_en eliminada si existía


In [5]:
try:
    conexion = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )
    # Crear un cursor
    cursor = conexion.cursor()
    # Consulta para seleccionar todos los datos de la tabla 'clientes'
    seleccionar_query = "SELECT * FROM platos;"
    
    cursor.execute(seleccionar_query)
    # Obtener todos los registros
    registros = cursor.fetchall()

    # Imprimir los registros
    for registro in registros:
        print(registro)

except Error as e:
    print(f"Error de conexión o durante la operación en la base de datos: {e}")

finally:

    if cursor:
        cursor.close()
    if conexion:
        conexion.close()

(0, 'DCCHamburguer', 'Hamburguesa con palta, tomate, mayonesa, rica para pasar la hora de almuerzo', True, 'hamburguesa', 'celiacos', 'hamburguesa, palta, tomate, mayonesa', 1, 10, True)
(1, 'Sushi', 'Una selecciÃ³n de rollitos de sushi tradicionales japoneses, servidos con wasabi y jengibre encurtido.', False, 'Internacional', 'pescetarianos', 'Arroz de sushi, algas nori, pescado crudo (como salmÃ³n, atÃºn o pez mantequilla), aguacate, pepino, cangrejo, vinagre de arroz, azÃºcar, sal', 3, 10, False)
(2, 'Pizza Margarita', 'Una pizza clÃ¡sica italiana con salsa de tomate, mozzarella fresca y albahaca fresca.', False, 'fast food', 'celiacos', 'Masa de pizza, salsa de tomate, mozzarella fresca, albahaca fresca, aceite de oliva.', 2, 32, True)
(3, 'Pizza Margarita', 'Una pizza clÃ¡sica italiana con salsa de tomate, mozzarella fresca y albahaca fresca.', True, 'fast food', 'celiacos', 'Masa de pizza, salsa de tomate, mozzarella fresca, albahaca fresca, aceite de oliva.', 2, 41, False)
(4, 

In [6]:
try:
    conexion = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )
    # Crear un cursor
    cursor = conexion.cursor()
    # Consulta para seleccionar todos los datos de la tabla 'clientes'
    seleccionar_query = "SELECT * FROM servido_en;"
    
    cursor.execute(seleccionar_query)
    # Obtener todos los registros
    registros = cursor.fetchall()

    # Imprimir los registros
    for registro in registros:
        print(registro)

except Error as e:
    print(f"Error de conexión o durante la operación en la base de datos: {e}")

finally:

    if cursor:
        cursor.close()
    if conexion:
        conexion.close()

('DCCRestaurant', 0, 5000)
('Burger Queen', 1, 16000)
('DCCRestaurant', 2, 7000)
('Papa Juan', 3, 16000)
('Just Pizza', 4, 16000)
('Doble Queso', 6, 16000)
('Doble Queso', 7, 16000)
('Chick-Fil-A', 8, 7000)
('Just Pizza', 9, 4000)
('Casiss', 10, 16000)
('DeliQuick', 11, 6000)
('Chick-Fil-A', 12, 19000)
('Krusty Krab', 13, 16000)
('Meet Burger', 14, 16000)
('DCCRestaurant', 15, 16000)
('Donald Mc', 16, 16000)
('Doble Queso', 17, 16000)
('DeliQuick', 18, 16000)
('Metro Sandwich', 19, 16000)
('Carlos Sr', 20, 16000)
('Burger Queen', 21, 16000)
('Comida Peruana', 22, 18000)
('Nolita', 23, 5000)
('Metro Sandwich', 24, 11000)
('Burger Queen', 26, 20000)
('Casiss', 27, 18000)
('Pollos Hermanos', 28, 18000)
('Papa Juan', 29, 17000)
('Central Perk', 30, 13000)
('Chum Bucket', 31, 5000)
('Indian Cuisine', 32, 20000)
('DCCRestaurant', 33, 13000)
('Pollos Hermanos', 34, 19000)
('DCCRestaurant', 37, 5000)
('Indian Cuisine', 38, 19000)
('Outback Steak', 39, 8000)
('Pollos Hermanos', 40, 11000)
('Kru